# Probabilistic Ciruits
Probabilistic Circuits (PCs) are a deep model that can represent joint probability distributions and which can answer many proababilistic queries efficiently (i.e. tractable). This notebook contains an example of how to use PCs to model a joint distribution using Sum Product Networks (SPNs) which are a concrete instantiation of PCs. We will perform density estimation, marginalization and conditioning (i.e. classification) using a single SPN learned over a dataset.

## Installing `spflow`
The python package `spflow` allows us to use SPNs in a simple way. However, the authors currently rewrite the package and don't maintain the current version actively anymore. Thus installing `spflow` can be a bit tricky to do. The best way to it at the moment is to clone the git repository, build it locally and instal it using `pip`.

The following instructions should install `spflow`:

1. `git clone https://github.com/SPFlow/SPFlow.git`
2. `cd SPFlow/src`
3. `bash create_pip_dist.sh`
4. `pip install dist/spflow-0.0.40-py3-none-any.whl`

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from spn.algorithms.LearningWrappers import learn_parametric, learn_classifier
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian
from spn.structure.Base import Context
from spn.algorithms.Sampling import sample_instances
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.Marginalization import marginalize
from numpy.random.mtrand import RandomState
from spn.algorithms.MPE import mpe

In [16]:
data = pd.read_csv('../data/credit-data/prepared_data.csv', index_col=0)
train_data, test_data = train_test_split(data.to_numpy(), test_size=0.3)

## Learning SPNs
We have loaded the credit dataset above which represents a classification problem. Hence we can use the `learn_classifier` function provided by `spflow`. This is a function coming with some special tricks to learn SPNs efficiently and returns a SPN representing a joint distribution over the dataset we pass in.

In [18]:
leaf_types = [Gaussian] * (data.shape[1] - 1) + [Categorical]
spn_classification = learn_classifier(train_data,
                       Context(parametric_types=leaf_types).add_domains(train_data),
                       learn_parametric, 17)

/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default 

## Performing Classification
Once we have obtained the SPN representing $p(\mathbf{x}, y)$ in our case, we have to compute the conditional using Bayes Rule: $p(y | \mathbf{x}) = \frac{p(\mathbf{x}, y)}{\int_{\mathbf{X}} p(\mathbf{x}, y)}$. In pratice we do an equivalent approach by providing evidence (i.e. the features we have) and ask for the value of $y$ maximizing the probability $p(\mathbf{x}, y)$. This gives us the class $y$ for each sample in the test set and is done as follows:

In [23]:
test_data_cp = np.copy(test_data) # copy data as mpe is inplace-operation
test_data_cp[:, -1] = np.nan # indicate columns to be classified
pred = mpe(spn_classification, test_data_cp)
y_pred = pred[:, -1]
y_true = test_data[:, -1]
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')

/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/spn/structure/leaves/parametric/Inference.py:88: RuntimeWarning: divide by zero encountered in log
  probs[idx_in] = np.array(np.log(node.p))[cat_data[~out_domain_ids]]


## Compute Density
We can compute the density (or log-denisty in this case) by simply calling the `log_likelihood`-function which evaluates a given SPN buttom-up given some data.

In [26]:
lls = log_likelihood(spn_classification, test_data)
lls

/home/jonas/anaconda3/envs/mldl/lib/python3.10/site-packages/spn/structure/leaves/parametric/Inference.py:88: RuntimeWarning: divide by zero encountered in log
  probs[idx_in] = np.array(np.log(node.p))[cat_data[~out_domain_ids]]


array([[-9.41287407e+01],
       [-8.94086615e+01],
       [-9.26705452e+01],
       [-1.01318214e+02],
       [-9.46862045e+01],
       [-8.63945342e+01],
       [-9.86381707e+01],
       [-8.23094666e+01],
       [-1.46883425e+02],
       [-4.74307152e+02],
       [-2.32038431e+02],
       [-9.37334622e+01],
       [-9.65681805e+01],
       [-9.02032729e+01],
       [-9.82859032e+01],
       [-9.73552251e+01],
       [-1.00003784e+02],
       [-8.37001400e+01],
       [-8.81550526e+01],
       [-8.97026260e+01],
       [-1.00179822e+02],
       [-9.35340132e+01],
       [-9.55084760e+01],
       [-9.65981349e+01],
       [-1.14606175e+02],
       [-9.14639387e+01],
       [-1.50664904e+02],
       [-1.15136330e+02],
       [-9.68978312e+01],
       [-8.40711556e+01],
       [-9.64223176e+01],
       [-8.49584996e+01],
       [-1.05380489e+02],
       [-9.43911467e+01],
       [-8.50819402e+01],
       [-1.01405734e+02],
       [-8.66420876e+01],
       [-9.35959138e+01],
       [-9.2

## Sampling from SPNs
As SPNs are generative models we can use them to sample new data from the distribution they represent. With `spflow` this can be simply done using the `sample_instances`-function:

In [33]:
nan_arr = np.array([[np.nan] * (data.shape[1])]*10)
samples = sample_instances(spn_classification, nan_arr, RandomState(123))
samples

array([[ 4.91335668e+01,  7.25043996e+04,  4.07042820e+03,
         1.65411756e+00,  4.80142738e+00, -9.94675788e+02,
        -2.24781965e+01,  5.49597447e+00,  2.14396037e+02,
        -2.38435583e+00,  3.40912312e+02,  1.30756701e+03,
         3.10732016e+01,  2.68439331e+01, -5.61844891e+03,
         3.63452661e+02,  3.84946055e+02,  1.00000000e+00],
       [ 2.93543819e+01,  5.70958873e+04,  4.72955962e+03,
         6.20368873e+00,  8.64749604e+00, -1.60867573e+03,
        -4.98934214e+00,  2.15406829e+01,  1.62705717e+01,
         1.13112213e+01,  1.48202012e+02,  2.31094500e+03,
         3.30404931e+01,  1.58375306e+01,  1.77436192e+02,
         3.95305967e+01,  6.34458982e+02,  1.00000000e+00],
       [ 2.43967557e+01,  7.47549901e+04,  4.18379504e+03,
         4.15947511e+00,  4.20618045e+00,  9.98325164e+02,
         5.35238035e+00,  2.94026250e+01, -1.44524568e+02,
         1.54236127e+01,  9.24269733e+01,  1.97527298e+03,
         3.36081595e+01,  1.41384309e+01,  9.90319427e

## Marginalization
To compute the marginal of a joint distribution (in this case we marginalize out $X_{10}$) we can us the `marginalize`-function which yields a new SPN representing the marginal distribution $\int_{X_{10}} p$

In [34]:
keep_inds = list(range(10)) + list(range(11, 19))
marg_spn = marginalize(spn_classification, keep_inds)
marg_spn

SumNode_0